In [ ]:
%pip install comet_ml #libreria a instalar

In [ ]:
import comet_ml
comet_ml.init(project_name="RNA_Densa_M2") #nombre a dar al experimento

In [ ]:
# Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist #base de datos
from tensorflow.keras.models import Sequential # checar cada uno de los que faltan
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint # sirve para dar respuesta del proceso de entrenamiento de la red

In [ ]:
experiment = comet_ml.Experiment(  # para activar las Graficas en comet
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True, # para subir el codigo a comet para no perder el progreso
)

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/nicglowss/rna-densa-m2/89cab9133f194c1ca9342db7397f32a9
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Reposit

In [ ]:
# pre-procesamiento
dataset=mnist.load_data()# en el dataset se guardan todas las imagenes del mnist con sus etiquetas
(x_train, y_train), (x_test, y_test) = dataset

# se van a guardar en el forato de binario
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# =numero de pixeles normalizando el no. de pixeles
x_train /= 255  # x_trainv = x_trainv/255
x_test /= 255

#clasificacion de clases
num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes) #analiza los datos y los acomoda en las distintas clases
y_testc = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
# Parametros
parameters = {
    "batch_size": 10,
    "epochs": 100, # Variamos las epocas de 30/10/50/100
    "optimizer": "SGD",
    "loss": "categorical_crossentropy",
}

experiment.log_parameters(parameters) # diccionario subido a comet

In [ ]:
#Arquitetura de la red
model = Sequential()
# aqui empezamos
model.add(Input(shape=(28,28))) #Flaten no tiene la opcion input_shape por lo tanto se tiene que agregar esta capa
                                #28x28 es el tamaño de la imagen
model.add(Flatten()) #Otra forma de aplanar las imagenes, las forma en una linea
model.add(Dense(784, activation='sigmoid')) # esta es la capa densa de no neuronas de forma vertical (en este caso agarramos el total de pixeles)
                                            # su funcion de activacion es la sigmoide (respuesta de la neurona de 0 a 1)
                                            # Estas dos son las principales

model.add(Dense(10, activation='softmax'))  # son las de salida literal porque estan al final

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 784)               615440    
                                                                 
 dense_1 (Dense)             (None, 10)                7850      
                                                                 
Total params: 623290 (2.38 MB)
Trainable params: 623290 (2.38 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# specify the path where you want to save the model
filepath = "mejor-modelo1.1.hdf5"

# initialize the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
# Inicia el entrenamiento de la red
model.compile(loss=parameters['loss'],optimizer=SGD(learning_rate=0.1,ema_momentum=0.9),metrics=['accuracy'])
model.fit(x_train, y_trainc,
                    batch_size=parameters['batch_size'],
                    epochs=parameters["epochs"],
                    verbose=1,
                    validation_data=(x_test, y_testc),
                    callbacks=[checkpoint])

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/100
5991/6000 [============================>.] - ETA: 0s - loss: 0.4341 - accuracy: 0.8722
Epoch 1: val_loss improved from inf to 0.26684, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 18s 3ms/step - loss: 0.4337 - accuracy: 0.8723 - val_loss: 0.2668 - val_accuracy: 0.9227
Epoch 2/100
  18/6000 [..............................] - ETA: 17s - loss: 0.2500 - accuracy: 0.9333

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5995/6000 [============================>.] - ETA: 0s - loss: 0.2459 - accuracy: 0.9282
Epoch 2: val_loss improved from 0.26684 to 0.21299, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 19s 3ms/step - loss: 0.2460 - accuracy: 0.9282 - val_loss: 0.2130 - val_accuracy: 0.9371
Epoch 3/100
5999/6000 [============================>.] - ETA: 0s - loss: 0.1813 - accuracy: 0.9479
Epoch 3: val_loss improved from 0.21299 to 0.16231, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 20s 3ms/step - loss: 0.1812 - accuracy: 0.9479 - val_loss: 0.1623 - val_accuracy: 0.9508
Epoch 4/100
5985/6000 [============================>.] - ETA: 0s - loss: 0.1434 - accuracy: 0.9580
Epoch 4: val_loss improved from 0.16231 to 0.12810, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 18s 3ms/step - loss: 0.1435 - accuracy: 0.9580 - val_loss: 0.1281 - val_accuracy: 0.9617
Epoch 5/100
5992/6000 [=======================

In [ ]:
experiment.log_model("MNIST1", "mejor-modelo1.1.hdf5")

{'web': 'https://www.comet.com/api/asset/download?assetId=cd794acf0fc0450abd9de7381f129e7d&experimentKey=5d7108c24a7b4372b270a828d97cff81',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=cd794acf0fc0450abd9de7381f129e7d&experimentKey=5d7108c24a7b4372b270a828d97cff81',
 'assetId': 'cd794acf0fc0450abd9de7381f129e7d'}

In [ ]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/nicglowss/rna-densa-m2/5d7108c24a7b4372b270a828d97cff81
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [100]                  : (0.8723333477973938, 1.0)
COMET INFO:     batch_accuracy [60000]          : (0.09090909361839294, 1.0)
COMET INFO:     batch_loss [60000]              : (2.9205930331954733e-06, 6.227663993835449)
COMET INFO:     epoch_duration [100]            : (15.753651162000097, 24.299163925999892)
COMET INFO:     loss [100]                      : (0.0009161510388366878, 0.43367666006088257)
COMET INFO:     val_accuracy [100]              : (0.9226999878883362, 0.9822999835014343)
COMET INFO:   